In [1]:
import tensorflow as tf
from tensorflow import keras
import pathlib
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,AveragePooling2D
from keras.layers import BatchNormalization
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import Model, Input
from keras.layers import Average
from typing import List
from keras.utils.version_utils import training


print(tf.config.list_physical_devices('CPU'))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# Загрузка датасета и связанных гиперпараметров

In [3]:
data_dir = pathlib.Path('datasets/images_center_10')

In [4]:
batch_size = 32
img_height = 20
img_width = 20
num_classes = 2

In [47]:
with tf.device('/device:CPU:0'):
  train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 5804 files belonging to 2 classes.
Using 4644 files for training.


In [48]:
with tf.device('/device:CPU:0'):
  val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 5804 files belonging to 2 classes.
Using 1160 files for validation.


In [49]:
class_names = train_ds.class_names
print(class_names)

['negative', 'positive']


In [50]:
with tf.device('/device:CPU:0'):
  AUTOTUNE = tf.data.AUTOTUNE

  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Определение моделей и методов для работы с ними

In [5]:
def create_model_cnn(input_shape=(img_height, img_width), num_classes=num_classes):
  model = Sequential([
  keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3), name='rescaling_1_1'),
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes)
  ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [6]:
def create_model_cnn_2(input_shape=(img_height, img_width), num_classes=num_classes):
    model = Sequential()
    model.add(keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3), name='rescaling_2_1'))
    model.add(Conv2D(32, kernel_size = (3,3), activation='relu', input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = (5,5), strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(num_classes, activation = "sigmoid"))
    model.compile(optimizer = 'adam' , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

In [7]:
def create_LeNet5(input_shape=(img_height, img_width), num_classes=num_classes):
    model = Sequential()
    model.add(keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3), name='rescaling_3_1'))
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape, padding="same"))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    model.add(Flatten())
    model.add(Dense(84, activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(optimizer =  'adam' , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

In [22]:
def train_model(model, epochs):
    with tf.device('/device:CPU:0'):
        history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs)
        return history

# Создание моделей

In [8]:
model_cnn = create_model_cnn()
model_cnn_2 = create_model_cnn_2()
model_LeNet = create_LeNet5()

# Обучение

In [53]:
history_cnn = train_model(model_cnn, 30)

Epoch 1/30
146/146 [==============================] - 3s 11ms/step - loss: 0.1112 - accuracy: 0.9658 - val_loss: 0.0316 - val_accuracy: 0.9922
Epoch 2/30
146/146 [==============================] - 1s 8ms/step - loss: 0.0301 - accuracy: 0.9922 - val_loss: 0.0368 - val_accuracy: 0.9836
Epoch 3/30
146/146 [==============================] - 1s 8ms/step - loss: 0.0270 - accuracy: 0.9929 - val_loss: 0.0251 - val_accuracy: 0.9922
Epoch 4/30
146/146 [==============================] - 1s 8ms/step - loss: 0.0232 - accuracy: 0.9935 - val_loss: 0.0184 - val_accuracy: 0.9948
Epoch 5/30
146/146 [==============================] - 1s 9ms/step - loss: 0.0199 - accuracy: 0.9944 - val_loss: 0.0153 - val_accuracy: 0.9957
Epoch 6/30
146/146 [==============================] - 1s 8ms/step - loss: 0.0158 - accuracy: 0.9959 - val_loss: 0.0143 - val_accuracy: 0.9974
Epoch 7/30
146/146 [==============================] - 1s 8ms/step - loss: 0.0157 - accuracy: 0.9963 - val_loss: 0.0146 - val_accuracy: 0.9974
Epoch

In [54]:
history_cnn_2 = train_model(model_cnn_2, 30)

Epoch 1/30
146/146 [==============================] - 5s 30ms/step - loss: 0.0625 - accuracy: 0.9815 - val_loss: 2.3908 - val_accuracy: 0.4914
Epoch 2/30
146/146 [==============================] - 5s 31ms/step - loss: 0.0244 - accuracy: 0.9938 - val_loss: 1.1074 - val_accuracy: 0.5793
Epoch 3/30
146/146 [==============================] - 5s 31ms/step - loss: 0.0264 - accuracy: 0.9910 - val_loss: 0.0830 - val_accuracy: 0.9716
Epoch 4/30
146/146 [==============================] - 4s 30ms/step - loss: 0.0220 - accuracy: 0.9931 - val_loss: 0.0173 - val_accuracy: 0.9922
Epoch 5/30
146/146 [==============================] - 4s 30ms/step - loss: 0.0163 - accuracy: 0.9942 - val_loss: 0.0189 - val_accuracy: 0.9957
Epoch 6/30
146/146 [==============================] - 4s 30ms/step - loss: 0.0145 - accuracy: 0.9946 - val_loss: 0.0328 - val_accuracy: 0.9888
Epoch 7/30
146/146 [==============================] - 4s 30ms/step - loss: 0.0131 - accuracy: 0.9950 - val_loss: 0.0088 - val_accuracy: 0.9966

In [55]:
history_LeNet = train_model(model_LeNet, 30)

Epoch 1/30
146/146 [==============================] - 2s 14ms/step - loss: 0.0881 - accuracy: 0.9714 - val_loss: 0.0549 - val_accuracy: 0.9810
Epoch 2/30
146/146 [==============================] - 2s 13ms/step - loss: 0.0368 - accuracy: 0.9892 - val_loss: 0.0350 - val_accuracy: 0.9940
Epoch 3/30
146/146 [==============================] - 2s 12ms/step - loss: 0.0329 - accuracy: 0.9916 - val_loss: 0.0238 - val_accuracy: 0.9914
Epoch 4/30
146/146 [==============================] - 2s 13ms/step - loss: 0.0279 - accuracy: 0.9931 - val_loss: 0.0316 - val_accuracy: 0.9888
Epoch 5/30
146/146 [==============================] - 2s 13ms/step - loss: 0.0237 - accuracy: 0.9940 - val_loss: 0.0224 - val_accuracy: 0.9922
Epoch 6/30
146/146 [==============================] - 2s 14ms/step - loss: 0.0227 - accuracy: 0.9944 - val_loss: 0.0196 - val_accuracy: 0.9931
Epoch 7/30
146/146 [==============================] - 2s 14ms/step - loss: 0.0211 - accuracy: 0.9959 - val_loss: 0.0197 - val_accuracy: 0.9948

In [56]:
model_cnn.save_weights('checkpoints/model_cnn_weights/checkpoint_cpu')
model_cnn_2.save_weights('checkpoints/model_cnn_2_weights/checkpoint_cpu')
model_LeNet.save_weights('checkpoints/model_LeNet_weights/checkpoint_cpu')

# Загрузка сохранённых моделей

In [10]:
model_cnn.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_cnn_weights/checkpoint_cpu')
model_cnn_2.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_cnn_2_weights/checkpoint_cpu')
model_LeNet.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_LeNet_weights/checkpoint_cpu')

# Создание и обучение моделей для определения вне главной диагонали

In [19]:
data_dir = pathlib.Path('datasets/images_10')

with tf.device('/device:CPU:0'):
  train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

  val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

  AUTOTUNE = tf.data.AUTOTUNE
  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 2732 files belonging to 2 classes.
Using 2186 files for training.
Found 2732 files belonging to 2 classes.
Using 546 files for validation.


In [11]:
model_cnn_uncentered = create_model_cnn()
model_cnn_2_uncentered = create_model_cnn_2()
model_LeNet_uncentered = create_LeNet5()

In [23]:
history_cnn_uncentered = train_model(model_cnn_uncentered, 30)

Epoch 1/30
69/69 [==============================] - 2s 9ms/step - loss: 0.0370 - accuracy: 0.9881 - val_loss: 1.6069e-07 - val_accuracy: 1.0000
Epoch 2/30
69/69 [==============================] - 1s 8ms/step - loss: 4.4247e-07 - accuracy: 1.0000 - val_loss: 1.4584e-07 - val_accuracy: 1.0000
Epoch 3/30
69/69 [==============================] - 1s 7ms/step - loss: 3.8832e-07 - accuracy: 1.0000 - val_loss: 1.4082e-07 - val_accuracy: 1.0000
Epoch 4/30
69/69 [==============================] - 1s 8ms/step - loss: 5.6046e-07 - accuracy: 1.0000 - val_loss: 1.3405e-07 - val_accuracy: 1.0000
Epoch 5/30
69/69 [==============================] - 1s 8ms/step - loss: 4.9838e-07 - accuracy: 1.0000 - val_loss: 1.2598e-07 - val_accuracy: 1.0000
Epoch 6/30
69/69 [==============================] - 1s 9ms/step - loss: 4.5218e-07 - accuracy: 1.0000 - val_loss: 1.1855e-07 - val_accuracy: 1.0000
Epoch 7/30
69/69 [==============================] - 1s 9ms/step - loss: 3.9030e-07 - accuracy: 1.0000 - val_loss: 1.

In [24]:
history_cnn_2_uncentered = train_model(model_cnn_2_uncentered, 30)

Epoch 1/30


C:\Users\vit\.conda\envs\HICT\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


69/69 [==============================] - 3s 27ms/step - loss: 0.0242 - accuracy: 0.9886 - val_loss: 0.0223 - val_accuracy: 1.0000
Epoch 2/30
69/69 [==============================] - 2s 27ms/step - loss: 1.1490e-04 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 3/30
69/69 [==============================] - 2s 28ms/step - loss: 4.4680e-05 - accuracy: 1.0000 - val_loss: 1.9045e-04 - val_accuracy: 1.0000
Epoch 4/30
69/69 [==============================] - 2s 26ms/step - loss: 3.0128e-05 - accuracy: 1.0000 - val_loss: 4.6225e-05 - val_accuracy: 1.0000
Epoch 5/30
69/69 [==============================] - 2s 29ms/step - loss: 2.2663e-05 - accuracy: 1.0000 - val_loss: 1.2205e-05 - val_accuracy: 1.0000
Epoch 6/30
69/69 [==============================] - 2s 30ms/step - loss: 2.5964e-05 - accuracy: 1.0000 - val_loss: 4.0743e-06 - val_accuracy: 1.0000
Epoch 7/30
69/69 [==============================] - 2s 28ms/step - loss: 3.3214e-05 - accuracy: 1.0000 - val_loss: 2.0687e-06 - v

In [25]:
history_LeNet_uncentered = train_model(model_LeNet_uncentered, 30)

Epoch 1/30
69/69 [==============================] - 1s 14ms/step - loss: 0.0390 - accuracy: 0.9854 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/30
69/69 [==============================] - 1s 12ms/step - loss: 2.1813e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/30
69/69 [==============================] - 1s 12ms/step - loss: 2.1813e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/30
69/69 [==============================] - 1s 12ms/step - loss: 2.1813e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/30
69/69 [==============================] - 1s 13ms/step - loss: 2.1813e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/30
69/69 [==============================] - 1s 12ms/step - loss: 2.1813e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/30
69/69 [==============================] - 1s 12ms/step - loss: 2.1813e-10 - accuracy: 1.0000 - val_l

In [41]:
model_cnn_uncentered.save_weights('checkpoints/model_cnn_weights/checkpoint_cpu_uncentered')
model_cnn_2_uncentered.save_weights('checkpoints/model_cnn_2_weights/checkpoint_cpu_uncentered')
model_LeNet_uncentered.save_weights('checkpoints/model_LeNet_weights/checkpoint_cpu_uncentered')

# Export all models

In [12]:
model_cnn_uncentered.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_cnn_weights/checkpoint_cpu_uncentered')
model_cnn_2_uncentered.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_cnn_2_weights/checkpoint_cpu_uncentered')
model_LeNet_uncentered.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_LeNet_weights/checkpoint_cpu_uncentered')

In [15]:
model_cnn.save("D:/Study/HICT/HICT_Patterns/exported_models/centered_first.keras")
model_cnn_2.save("D:/Study/HICT/HICT_Patterns/exported_models/centered_second.keras")
model_LeNet.save("D:/Study/HICT/HICT_Patterns/exported_models/centered_third.keras")

model_cnn_uncentered.save("D:/Study/HICT/HICT_Patterns/exported_models/final_first.keras")
model_cnn_2_uncentered.save("D:/Study/HICT/HICT_Patterns/exported_models/final_second.keras")
model_LeNet_uncentered.save("D:/Study/HICT/HICT_Patterns/exported_models/final_third.keras")